In [65]:
# %pip install -qU langchain-community pypdf

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import fitz  # PyMuPDF
import os
import hashlib
import base64
from openai import OpenAI

In [3]:
client = OpenAI()

In [4]:
def encode_image(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")
        
        
def extract_text_and_images(pdf_path, output_folder, min_width=100, min_height=100, min_filesize=5000):
    """
    Extract text and unique images from each page of a PDF into a dictionary.
    Skips small icons/logos and duplicate images (by size + dimensions).
    """
    doc = fitz.open(pdf_path)
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    seen = set()  # store (width, height, filesize) triples
    results = {}  # dictionary to hold page data
    
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        page_dict = {"text": "", "images": []}
        
        # --- Extract text ---
        text = page.get_text("text").strip()
        page_dict["text"] = text if text else ""
        
        # --- Extract images ---
        for img_index, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            width, height = img[2], img[3]
            
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            filesize = len(image_bytes)
            
            # Skip icons/logos (small dimension or tiny file size)
            if width < min_width or height < min_height or filesize < min_filesize:
                continue
            
            # Skip duplicates based on (dimensions + file size)
            key = (width, height, filesize)
            if key in seen:
                continue
            seen.add(key)
            
            # Save unique image
            output_path = os.path.join(
                output_folder, f"page_{page_num+1}_img_{img_index+1}.{image_ext}"
            )
            with open(output_path, "wb") as f:
                f.write(image_bytes)
            
            page_dict["images"].append(output_path)
        
        results[page_num + 1] = page_dict
    
    doc.close()
    return results

In [5]:
pdf_data = extract_text_and_images("/home/jovyan/projects/data/helpkasra.pdf", "extracted_images")

# Print summary
for page, content in pdf_data.items():
    print(f"\n--- Page {page} ---")
    print("Text:", content["text"][:200], "...")  # preview first 200 chars
    print("Images:", content["images"])

    break


--- Page 1 ---
Text: 1 
 
 
 
 
 
 
 
 راهنمای
عمومی سامانه حضور و غیاب کسرا 
 
)(تردد ...
Images: ['extracted_images/page_1_img_1.jpeg', 'extracted_images/page_1_img_2.jpeg']


In [7]:
item = pdf_data[4]

content = [
    {"type": "text", "text": f"لطفاً این متن و تصاویر همراه آن را بررسی کن و **توضیح کامل بده**، بدون هیچ مقدمه‌ای:\n\n{item['text']}"}
]

for img in item["images"]:
    b64 = encode_image(img)
    content.append({
        "type": "image_url",
        "image_url": {
            "url": f"data:image/png;base64,{b64}"
        }
    })

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "تو یک دستیار هوشمند هستی که متن و تصویر را بررسی و توضیح می‌دهی، بدون هیچ جملهٔ اضافی یا مقدمه."},
        {"role": "user", "content": content}
    ]
)

print(response.choices[0].message.content)

متن و تصویر ارائه شده، به توضیحاتی در مورد یک نرم‌افزار سیستم مدیریت و عملکرد آن می‌پردازد. 

### توضیحات متن:

1. **کسرای من**: این بخش به عنوان صفحه اصلی یا میزبانی شما در نرم‌افزار اشاره دارد.

2. **نوار ابزار عمودی**: در این نوار ابزار، آیکن‌هایی وجود دارد که به صفحات مختلف دسترسی می‌دهد. با کلیک بر روی هر آیکن، نام صفحه نمایش داده می‌شود. برای مشاهده نام تمام صفحات، کافیست بر روی علامت مشخص شده کلیک کنید.

3. **پیام‌های ارسالی مدیر سیستم**: در این صفحه، تمامی پیام‌های ارسال شده توسط مدیر سیستم و همچنین صفحات پرکاربرد انتخاب شده شما بصورت مجزا قرار دارد.

4. **تنظیمات میزکار**: کاربران می‌توانند میزکار خود را به دلخواه تنظیم کنند و توضیحات اضافی درباره این ویژگی ارائه شده است.

### توضیحات تصویر:

- **انتخاب صفحات**: تصویر شامل یک نوار ابزار در سمت چپ که آیکن‌های مختلف برای دسترسی به صفحات است.
- **پیام‌های مدیر سیستم**: بخشی برای نمایش پیام‌هایی که مدیر ارسال کرده و قابلیت مشاهده پیام‌های قبلی و بعدی.
- **وضعیت مجوزها**: نموداری که وضعیت مجوزها و تعداد آن‌ها را نمایش می‌دهد.
- **ک

In [18]:
item = pdf_data[7]

content = [
    {"type": "text", "text": f"لطفاً این متن و تصاویر همراه آن را بررسی کن و **توضیح کامل بده**، بدون هیچ مقدمه‌ای:\n\n{item['text']}"}
]

for img in item["images"]:
    b64 = encode_image(img)
    content.append({
        "type": "image_url",
        "image_url": {
            "url": f"data:image/png;base64,{b64}"
        }
    })

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "تو یک دستیار هوشمند هستی که متن و تصویر را بررسی و توضیح می‌دهی، بدون هیچ جملهٔ اضافی یا مقدمه."},
        {"role": "user", "content": content}
    ]
)

print(response.choices[0].message.content)

متن ارائه شده راهنمایی برای جستجوی صفحات در یک سیستم نرم‌افزاری است. در اولین بخش توضیح داده شده که چطور می‌توان با کلیک بر روی آیکن جستجو در پنل سمت راست، کادر جستجو را فعال کرد. سپس کاربران می‌توانند بخشی از نام صفحه مورد نظر را تایپ کنند تا لیست صفحاتی که شامل آن نام هستند و دسترسی دارند، مشاهده کنند.

در ادامه بیان شده که زیر نام صفحه مورد نظر، مسیر دستیابی از طریق منو نمایش داده می‌شود. همچنین، استفاده از کلید ترکیبی SHIFT+F برای دسترسی سریع به کادر جستجو در کلیه صفحات نرم‌افزار اشاره شده است.

تصویر ارائه شده به نمایش رابط کاربری نرم‌افزار می‌پردازد. آیکن جستجو در سمت راست مشخص شده و قسمت‌های مختلف پنل کاربری نیز قابل مشاهده است، از جمله لیست درخواست‌ها و نمودارهایی که به تجزیه و تحلیل داده‌ها کمک می‌کند.
